In [ ]:
%%capture 
!pip install -r requirements.txt
!pip install torch torchvision

In [1]:
from ddpg_torch import AgentDDPG
from sac_torch import AgentSAC
# 
import torch
import gym
# from SAC import SAC_Agent
from ReplayBuffer import RandomBuffer, device
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os, shutil
import argparse
from Adapter import *
#
import gym 
import numpy as np
from utils import plotLearning 

import glob
from PIL import Image

from utils import show_video, convert_gif
from gym.wrappers.monitoring.video_recorder import VideoRecorder

    
from base64 import b64encode

In [3]:
# name_env = 'LunarLanderContinuous-v2'
name_env ="BipedalWalker-v3"
env = gym.make(name_env)

print([*env.observation_space.shape],*env.action_space.shape)

input_dims = [*env.observation_space.shape]
n_actions = env.action_space.shape[0]

print(input_dims,n_actions)


def make_gif(frame_folder,name):
      frames = [Image.open(image) for image in glob.glob(f"{frame_folder}/*.png")]
      frame_one = frames[0]
      frame_one.save(os.path.join(frame_folder,f"{name}.gif"), format="GIF", append_images=frames,
                save_all=True, duration=100, loop=0)    
def render_mp4(videopath: str) -> str:
    """
    Gets a string containing a b4-encoded version of the MP4 video
    at the specified path.
    """
    mp4 = open(videopath, 'rb').read()
    base64_encoded_mp4 = b64encode(mp4).decode()
    return f'<video width=400 controls><source src="data:video/mp4;' \
          f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

[24] 4
[24] 4


#### DDPG

In [ ]:
agentDDPG = AgentDDPG(
    alpha=0.0001,
    beta=0.00001,
    input_dims=input_dims,
    tau=0.001,env=env,
    rollout_len = 500,
    total_rollouts = 1000,
    batch_size=64,
    layer1_size=256,
    layer2_size=128,
    n_actions=n_actions,
    agent_dir=os.path.join(os.getcwd(),name_env))
    

In [ ]:
agentDDPG.train(True,plot_save=os.path.join(os.getcwd(),'plots','DDPG',name_env))

In [ ]:
make_gif(os.path.join(os.getcwd(),'plots','DDPG',name_env),name_env)

#### SAC

In [ ]:
!pip install -U pip
!pip install tensorflow
!pip install tensorboar

In [4]:
kwargs = {
        "state_dim": input_dims[0],
        "action_dim": n_actions,
        "gamma": .99,
        "hid_shape": (256,128),
        "a_lr": 3e-5,
        "c_lr": 3e-5,
        "batch_size":128,
        "alpha":0.12,
        "adaptive_alpha":True
    }

agentSAC = AgentSAC(**kwargs, agent_dir=os.path.join(os.getcwd(),name_env,'SAC'))

if not os.path.exists(os.path.join(os.getcwd(),name_env,'SAC')): 
    os.mkdir(os.path.join(os.getcwd(),name_env,'SAC'))
BriefEnvName = ['BWv3', 'BWHv3', 'Lch_Cv2', 'PV0', 'Humanv2', 'HCv2']

EnvIdex = 0
    
random_seed = 1

The new directory is created!


In [5]:
timenow = str(datetime.now())[0:-10]
timenow = ' ' + timenow[0:13] + '_' + timenow[-2::]
writepath = os.path.join(os.getcwd(),name_env,'SAC','runs',f'SAC_{BriefEnvName[EnvIdex]}' + timenow)
if os.path.exists(writepath): 
    shutil.rmtree(writepath)
writer= SummaryWriter(log_dir=writepath)
print(f'writepath: {writepath}')

writepath: c:\Users\Dmitry\Desktop\Lab5\GreenHouse\BipedalWalker-v3\SAC\runs\SAC_BWv3 2023-09-20 19_04


In [6]:
%load_ext tensorboard
agentSAC.train(
    env=env,total_steps=int(5e6),EnvIdex = 0,write=True,
    writer= writer,
    plot=True,
    plot_path=os.path.join(os.getcwd(),name_env,'SAC','plots'))

update_every = 50;eval_interval = 1000;save_interval = 2500;max_action = 1.0steps_per_epoch = 1600;start_steps = 8000;update_after = 3200


c:\Users\Dmitry\Desktop\Lab5\GreenHouse\.venv\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


KeyboardInterrupt: 

In [ ]:
make_gif(os.path.join(os.getcwd(),name_env,'SAC','plots'),name_env)

In [ ]:
from tensorboard.backend.event_processing import event_accumulator

ea = event_accumulator.EventAccumulator(writepath,
       size_guidance={ # see below regarding this argument
       event_accumulator.COMPRESSED_HISTOGRAMS: 500,
       event_accumulator.IMAGES: 4,
       event_accumulator.AUDIO: 4,
       event_accumulator.SCALARS: 0,
       event_accumulator.HISTOGRAMS: 1,
       })
ea.Reload() # loads events from file
# Out[3]: <tensorflow.python.summary.event_accumulator.EventAccumulator at 0x7fdbe5ff59e8>
ea.Tags() 
ea.Scalars('Loss')

#### OLD

In [ ]:
!pip freeze > requirements.txt

In [ ]:
from pyvirtualdisplay import Display
from selenium import webdriver

display = Display(visible=0, size=(800, 600))
display.start()

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')  # May be required in some environments
chrome_options.add_argument('--disable-dev-shm-usage')  # May be required in some environments
chrome_options.add_argument('--headless')

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# videos_dir = os.path.join(f"{name_env}",f"{name_env}_before_training.mp4")
# # if not os.path.exists(videos_dir):
# #     # Create a new directory because it does not exist
# #     os.makedirs(videos_dir)
# #     print("The new directory is created!")
    
# before_training = videos_dir

# video = VideoRecorder(env, before_training)
# # returns an initial observation
# env.reset()
# for i in range(200):
#   env.render()
#   video.capture_frame()
#   # env.action_space.sample() produces either 0 (left) or 1 (right).
#   new_state,reward,done,info,_ =  env.step(env.action_space.sample())
#   # Not printing this time
#   #print("step", i, observation, reward, done, info)

# video.close()
# env.close()

before_training = "before_training.mp4"

video = VideoRecorder(env, before_training)
# returns an initial observation
env.reset()
for i in range(200):
  env.render()
  video.capture_frame()
  # env.action_space.sample() produces either 0 (left) or 1 (right).
  act = env.action_space.sample() 
  observation, reward, done, info,_ = env.step(act)
  # Not printing this time
  #print("step", i, observation, reward, done, info)

video.close()
env.close()

In [ ]:
from IPython.display import HTML
html = render_mp4(before_training)
HTML(html)

In [ ]:
if not os.path.exists(plot_save):
    # Create a new directory because it does not exist
    os.makedirs(plot_save)
    print("The new directory is created!")
for i in range(1000):
    done = False
    score = 0
    obs = env.reset()[0]
    # print(obs)
    while not done:
        act = agent.choose_action(obs) 
        new_state,reward,done,info,_ = env.step(act)
        agent.remember(state=obs,action=act,reward=reward,new_state=new_state,done=done)
        
        agent.learn()
        score += reward
        obs = new_state
    score_history.append(score)
    print(f'Episode {i} score {round(score, 2)} 100 game average {round(np.mean(score_history[-100:]), 2)}')
    if i % 25 == 0:
        agent.save_models()   
    filename = f'{i}.png'
    plotLearning(score_history,os.path.join(plot_save,filename),window=100)
make_gif(plot_save,name_env)

In [ ]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder
before_training = "before_training.mp4"

video = VideoRecorder(env, before_training)
# returns an initial observation
env.reset()
for i in range(200):
  env.render()
  video.capture_frame()
  # env.action_space.sample() produces either 0 (left) or 1 (right).
  observation, reward, done, info = env.step(env.action_space.sample())
  # Not printing this time
  #print("step", i, observation, reward, done, info)

video.close()
env.close()

In [ ]:
for env_name in [
    'LunarLanderContinuous-v2',
    "BipedalWalker-v3"]:
    env = gym.make(env_name)
    input_dims = [*env.observation_space.shape]
    n_actions = env.action_space.shape[0]   
    
    agent = AgentDDPG(
    alpha=0.000025,
    beta=0.00025,
    input_dims=input_dims,
    tau=0.001,env=env,
    batch_size=64,
    layer1_size=400,
    layer2_size=300,
    n_actions=n_actions,
    agent_dir=os.path.join(os.getcwd(),env_name))
    
    
    
    np.random.seed(42)
    score_history = []
    plot_save = os.path.join(os.getcwd(),'plots',env_name)
    
    
    if not os.path.exists(plot_save):
        # Create a new directory because it does not exist
        os.makedirs(plot_save)
        print("The new directory is created!")
    for i in range(1000):
        done = False
        score = 0
        obs = env.reset()[0]
        # print(obs)
        while not done:
            act = agent.choose_action(obs) 
            new_state,reward,done,info,_ = env.step(act)
            agent.remember(state=obs,action=act,reward=reward,new_state=new_state,done=done)
            agent.learn()
            score += reward
            obs = new_state
        score_history.append(score)
        print(f'Episode {i} score {round(score, 2)} 100 game average {round(np.mean(score_history[-100:]), 2)}')
        if i % 25 == 0:
            agent.save_models()   
        filename = f'{i}.png'
        plotLearning(score_history,os.path.join(plot_save,filename),window=100)
    make_gif(plot_save,env_name)